In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/working/__notebook__.ipynb


In [2]:
import os
os.getcwdb()

b'/kaggle/working'

In [3]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("kate")

import wandb
import os
os.system("wandb login 525037aaae74525566dca1246bce266cc1c1330b")

import wandb
wandb.init(project="emotion_model", entity="catherine1015")

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: catherine1015. Use `wandb login --relogin` to force relogin


In [4]:
import os
os.chdir('/kaggle/input/weibodata')

In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.8/136.8 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 837.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.6/484.6 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.3/500.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import os
import json
import torch
import shutil
import argparse
from loguru import logger
import numpy as np
from torch.cuda import amp
from tqdm import tqdm
from torch.utils.data import Dataset
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


In [8]:
class WBDataset(Dataset):
    def __init__(self, path, tokenizer='bert-base-chinese'):
        self.data_list = self._read_file(path)
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer)
        self.label = {'happy': 0, 'angry': 1, 'sad': 2, 'fear': 3, 'surprise': 4, 'neutral': 5}

    def _read_file(self, path):
        with open(path, 'r') as f:
            data_list = json.load(f)
        return data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        data = self.data_list[index]
        content = data['content']
        label = self.label[data['label']]
        token = self.tokenizer(content, padding='max_length', truncation=True, max_length=140)
        input_ids = token['input_ids']
        token_type_ids = token['token_type_ids']
        attention_mask = token['attention_mask']
        return input_ids, token_type_ids, attention_mask, label

    @staticmethod
    def collate_fn(batch):
        input_ids, token_type_ids, attention_mask, label = list(zip(*batch))
        input_ids = torch.tensor(input_ids)
        token_type_ids = torch.tensor(token_type_ids)
        attention_mask = torch.tensor(attention_mask)
        label = torch.tensor(label)
        return input_ids, token_type_ids, attention_mask, label


In [9]:
class Trainer():
    def __init__(self, train_dataloader, model, lr, epochs, output_dir, val_dataloader=None, save_model_iter=5):
        self.device = torch.device(device) #'cuda:0'
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.n_batch = len(train_dataloader)
        self.model = model.to(self.device)
        self.lr = lr
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)
        self.num_training_steps = self.n_batch * epochs
        self.scheduler = get_linear_schedule_with_warmup(optimizer=self.optimizer,
                                                         num_training_steps=self.num_training_steps,
                                                         num_warmup_steps=self.n_batch)
        self.scaler = amp.GradScaler(enabled=True)
        self.epochs = epochs
        self.start_epoch = 0
        self.global_step = 0
        self.save_model_iter = save_model_iter
        self.output_dir = output_dir
        self._init_logger()

        self.writer = SummaryWriter(os.path.join(self.output_dir, 'tensorboard'))
        self.metrics = {'val_loss': 0, 'val_acc': 0, 'val_f1_score': 0}

    def _init_logger(self):
        log_file_name = 'train_{time}.log'
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
        log_path = os.path.join(self.output_dir, log_file_name)
        logger.add(log_path, rotation='1 week', retention='30 days', enqueue=True)

    def train_epoch(self, epoch):
        self.model.train()
        pbar = tqdm(enumerate(self.train_dataloader), total=self.n_batch)
        train_loss = AverageMeter('Loss', ':6.3f')
        acc = AverageMeter('acc', ':6.3f')
        f1 = AverageMeter('f1_score', ':6.3f')

        for index, (input_ids, token_type_ids, attention_mask, label) in pbar:
            self.global_step += 1
            output = model(input_ids=input_ids.to(self.device), attention_mask=attention_mask.to(self.device),
                           labels=label.to(self.device))
            loss = output.loss
            logits = output.logits

            self.scaler.scale(loss).backward()
            self.scaler.step(self.optimizer)  # optimizer.step
            self.scaler.update()
            self.optimizer.zero_grad()
            self.scheduler.step()

            preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
            train_f1_score = f1_score(y_true=label.cpu().numpy(), y_pred=preds, average='macro')
            train_acc = accuracy_score(y_true=label.cpu().numpy(), y_pred=preds)
            train_loss.update(loss.detach().cpu().numpy().tolist())
            acc.update(train_acc)
            f1.update(train_f1_score)
            pbar.set_description(f'{train_loss.avg: .3f}  {acc.avg: .3f}  {f1.avg: .3f}')
            pbar.set_postfix({'epoch': str(epoch) + '/' + str(self.epochs),
                              'lr': self.optimizer.state_dict()['param_groups'][0]['lr']})

            self.writer.add_scalar('Train/loss', train_loss.avg, self.global_step)
            self.writer.add_scalar('Train/acc', acc.avg, self.global_step)
            self.writer.add_scalar('Train/f1_score', f1.avg, self.global_step)
            self.writer.add_scalar('Train/lr', self.optimizer.state_dict()['param_groups'][0]['lr'],
                                   self.global_step)

        logger.info(f'{train_loss.avg: .3f}  {acc.avg: .3f}  {f1.avg: .3f}')

    def on_epoch_finish(self, epoch):
        save_best = False
        metric = None
        if self.val_dataloader is not None:
            with torch.no_grad():
                val_loss, val_acc, val_f1_score = self._val()
            self.writer.add_scalar('Val/loss', val_loss, epoch)
            self.writer.add_scalar('Val/acc', val_acc, epoch)
            self.writer.add_scalar('Val/f1_score', val_f1_score, epoch)
            metric = {'val_loss': val_loss, 'val_acc': val_acc, 'val_f1_score': val_f1_score}
            logger.info(f'complete {epoch} epochs, val result: {metric}')
            print('----val_f1_score, val_f1_score',val_f1_score, self.metrics['val_f1_score'] )
            if val_f1_score > self.metrics['val_f1_score']:
                save_best = True
                print('-------save_best', save_best)
                self.metrics = {'val_loss': val_loss, 'val_acc': val_acc, 'val_f1_score': val_f1_score}

        if save_best:
            self._save_checkpoint(epoch, metric, save_best)

        elif epoch % self.save_model_iter == 0:
            self._save_checkpoint(epoch, metric)
        else:
            pass

    def _save_checkpoint(self, epoch, metric, save_best=False):

        state_dict = self.model.state_dict()
        state = {
            'epoch': epoch,
            'metric': metric,
            'state_dict': state_dict,
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict(),
        }

        name = f'{epoch}.pt'
        path = os.path.join(self.output_dir, name)
        torch.save(state, path, _use_new_zipfile_serialization=False)
        if save_best:
            shutil.copy(path, os.path.join(self.output_dir, 'best.pt'))
            logger.info(f'Saving current best: {path}')
        else:
            logger.info(f'Saving checkpoint: {path}')

    def train(self):
        logger.info('===============================START===============================')
        for epoch in range(self.start_epoch + 1, self.epochs + 1):
            print('\n loss    acc    f1_score')
            self.train_epoch(epoch)
            self.on_epoch_finish(epoch)
        logger.info(f'complete the training, best result: {self.metrics}')
        logger.info('===============================END===============================')

    def _val(self):
        self.model.eval()
        pbar = tqdm(enumerate(self.val_dataloader), total=len(self.val_dataloader))

        preds = []
        labels = []
        losses = []
        for index, (input_ids, token_type_ids, attention_mask, label) in pbar:
            output = self.model(input_ids=input_ids.to(self.device), attention_mask=attention_mask.to(self.device),
                                labels=label.to(self.device))

            loss = output.loss
            logits = output.logits
            losses.append(loss.detach().cpu().numpy().tolist())
            preds.extend(np.argmax(logits.detach().cpu().numpy(), axis=1).tolist())
            labels.extend(label.cpu().numpy().tolist())
        val_f1_score = f1_score(y_true=np.array(labels), y_pred=np.array(preds), average='macro')
        val_acc = accuracy_score(y_true=np.array(labels), y_pred=np.array(preds))
        val_loss = np.array(losses).mean()
        return val_loss, val_acc, val_f1_score


In [10]:
def parse_args():
    parser = argparse.ArgumentParser(description='Bert fine-tune on SMP2020-EWECT Dataset')
    parser.add_argument('--model_name', default='bert-base-chinese', type=str,help='huggingface transformer model name')
    parser.add_argument('--num_labels', default=6, type=int, help='fine-tune num labels')
    parser.add_argument('--train_data_path', default='/kaggle/input/weibodata1/clean/usual_train.txt', type=str, help='train data path')
    parser.add_argument('--val_data_path', default='/kaggle/input/weibodata1/clean/usual_eval_labeled.txt', type=str,help='train data path')
    parser.add_argument('--batch_size', default=64, type=int, help='train and validation batch size')
    parser.add_argument('--dataloader_num_workors', default=8, type=int, help='pytorch dataloader num workers')
    parser.add_argument('--lr', default=1e-5, type=float, help='train learning rate')
    parser.add_argument('--epochs', default=30, type=int, help='train epochs')
    parser.add_argument('--output_dir', default='/kaggle/working/', type=str, help='save dir')
    parser.add_argument('--save_model_iter', default=5, type=int, help='save model num epochs on training')
    args = parser.parse_args(args=[])
    return args


In [11]:
args = parse_args()
print('---args', args)

---args Namespace(batch_size=64, dataloader_num_workors=8, epochs=30, lr=1e-05, model_name='bert-base-chinese', num_labels=6, output_dir='/kaggle/working/', save_model_iter=5, train_data_path='/kaggle/input/weibodata1/clean/usual_train.txt', val_data_path='/kaggle/input/weibodata1/clean/usual_eval_labeled.txt')


In [12]:
model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=args.num_labels)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
#print(model)
train_dataset = WBDataset(args.train_data_path, args.model_name)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True,
                              num_workers=args.dataloader_num_workors,
                              collate_fn=WBDataset.collate_fn)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
val_dataset = WBDataset(args.val_data_path, args.model_name)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=True,
                            num_workers=args.dataloader_num_workors,
                            collate_fn=WBDataset.collate_fn, drop_last=False)
trainer = Trainer(train_dataloader=train_dataloader, model=model, lr=args.lr, epochs=args.epochs,
                  output_dir=args.output_dir, save_model_iter=args.save_model_iter, val_dataloader=val_dataloader)
trainer.train()

print('-----train finished')


2022-09-02 09:59:51.341 | INFO     | __main__:train:116 - ===============================START===============================



 loss    acc    f1_score


 1.172   0.565   0.440: 100%|██████████| 434/434 [05:49<00:00,  1.24it/s, epoch=1/30, lr=1e-5]
2022-09-02 10:05:41.669 | INFO     | __main__:train_epoch:68 -  1.172   0.565   0.440
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 32/32 [00:08<00:00,  3.63it/s]
2022-09-02 10:05:50.876 | INFO     | __main__:on_epoch_finish:80 - complete 1 epochs, val result: {'val_loss': 0.661243581213057, 'val_acc': 0.771657486229344, 'val_f1_score': 0.7368500026869432}


----val_f1_score, val_f1_score 0.7368500026869432 0
-------save_best True


2022-09-02 10:05:54.664 | INFO     | __main__:_save_checkpoint:111 - Saving current best: /kaggle/working/1.pt


...

----val_f1_score, val_f1_score 0.7404462685220438 0.7687895832953949

 loss    acc    f1_score


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 0.012   0.995   0.992: 100%|██████████| 434/434 [05:50<00:00,  1.24it/s, epoch=30/30, lr=0]
2022-09-02 12:59:44.173 | INFO     | __main__:train_epoch:68 -  0.012   0.995   0.992
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, l

----val_f1_score, val_f1_score 0.7422494731255415 0.7687895832953949


2022-09-02 12:59:55.011 | INFO     | __main__:_save_checkpoint:113 - Saving checkpoint: /kaggle/working/30.pt
2022-09-02 12:59:55.014 | INFO     | __main__:train:121 - complete the training, best result: {'val_loss': 0.6026068404316902, 'val_acc': 0.7926890335503255, 'val_f1_score': 0.7687895832953949}
2022-09-02 12:59:55.016 | INFO     | __main__:train:122 - ===============================END===============================


-----train finished
